# Section 1


## Part 1
```
Implement a content-based filtering recommendation system using the MovieLens dataset.
```

import libraries and dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

movies = pd.read_csv('data/movie.csv')
ratings = pd.read_csv('data/rating.csv')
movies.head(10)
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
5,1,112,3.5,2004-09-10 03:09:00
6,1,151,4.0,2004-09-10 03:08:54
7,1,223,4.0,2005-04-02 23:46:13
8,1,253,4.0,2005-04-02 23:35:40
9,1,260,4.0,2005-04-02 23:33:46


preprocessing

In [2]:
# For simplicity, let's use the movie genres as features
movies['genres'] = movies['genres'].str.split('|')

# Create a new DataFrame with movieId and title
movie_titles = movies[['movieId', 'title']].drop_duplicates()

# Drop the title column from the movies DataFrame to avoid conflicts during merge
movies = movies.drop(columns=['title'])

# Explode the genres and create dummy variables
movies = movies.explode('genres')
movies = pd.get_dummies(movies, columns=['genres'])

# Group by movieId and sum the genre dummy variables
movies = movies.groupby('movieId').sum().reset_index()

# Merge the movie titles back into the movies DataFrame
movies = movies.merge(movie_titles, on='movieId')

# Ensure the title column is present
print(movies.head())


   movieId  genres_(no genres listed)  genres_Action  genres_Adventure  \
0        1                          0              0                 1   
1        2                          0              0                 1   
2        3                          0              0                 0   
3        4                          0              0                 0   
4        5                          0              0                 0   

   genres_Animation  genres_Children  genres_Comedy  genres_Crime  \
0                 1                1              1             0   
1                 0                1              0             0   
2                 0                0              1             0   
3                 0                0              1             0   
4                 0                0              1             0   

   genres_Documentary  genres_Drama  ...  genres_Horror  genres_IMAX  \
0                   0             0  ...              0            0

calculate cosine similarity between movies

In [3]:
movie_features = movies.drop(columns=['movieId', 'title'])
similarity_matrix = cosine_similarity(movie_features)

Function to recommend movies based on content similarity

In [4]:
def recommend_movies(movie_id, similarity_matrix, movies, top_n=10):
    movie_index = movies[movies['movieId'] == movie_id].index[0]
    similarity_scores = list(enumerate(similarity_matrix[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:top_n+1]
    movie_indices = [i[0] for i in similarity_scores]
    recommended_movies = movies.iloc[movie_indices][['movieId', 'title']]
    return recommended_movies

example

In [5]:
movie_id = 1
movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
print(f"Recommendations for Movie: {movie_title}")
recommendations = recommend_movies(movie_id, similarity_matrix, movies)
print(f"Recommendations for Movie ID {movie_id}:")
print(recommendations)

Recommendations for Movie: Toy Story (1995)
Recommendations for Movie ID 1:
       movieId                                              title
2209      2294                                        Antz (1998)
3027      3114                                 Toy Story 2 (1999)
3663      3754     Adventures of Rocky and Bullwinkle, The (2000)
3922      4016                   Emperor's New Groove, The (2000)
4790      4886                              Monsters, Inc. (2001)
10114    33463  DuckTales: The Movie - Treasure of the Lost La...
10987    45074                                   Wild, The (2006)
11871    53121                             Shrek the Third (2007)
13337    65577                     Tale of Despereaux, The (2008)
18274    91355  Asterix and the Vikings (Astérix et les Viking...


## Part 2
```
Develop a hybrid recommendation system that combines user-based collaborative filtering and content-based filtering.
```

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# Load and preprocess data
ratings = pd.read_csv('data/rating.csv')
movies = pd.read_csv('data/movie.csv')

# Subset the data (optional for performance)
ratings_subset = ratings.sample(frac=0.05, random_state=42)

# Create user-item matrix for ALS
user_item_matrix = ratings_subset.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_item_sparse = csr_matrix(user_item_matrix.values)

# Train ALS model
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=15)
als_model.fit(user_item_sparse.T)  # Note: Transpose for ALS

# Process movies for content-based filtering
movies['genres'] = movies['genres'].str.split('|')
movies = movies.explode('genres')
movies = pd.get_dummies(movies, columns=['genres'])
movies = movies.groupby('movieId').sum().reset_index()  # This line groups by 'movieId' and sums the columns
movie_features = movies.drop(columns=['title'])  # Drop 'title' only if needed

# Calculate movie similarity matrix
movie_similarity_matrix = cosine_similarity(movie_features.iloc[:, 1:])

# Map user IDs and movie IDs to indices
user_id_mapping = {user_id: idx for idx, user_id in enumerate(user_item_matrix.index)}
reverse_user_id_mapping = {idx: user_id for user_id, idx in user_id_mapping.items()}

movie_id_mapping = {movie_id: idx for idx, movie_id in enumerate(user_item_matrix.columns)}
reverse_movie_id_mapping = {idx: movie_id for movie_id, idx in movie_id_mapping.items()}

c:\Python311\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.02572154998779297 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
# Recommendation functions
def recommend_movies_cf(user_id, als_model, user_item_sparse, top_n=10):
    """
    Recommend movies using ALS collaborative filtering.
    """
    if user_id not in user_id_mapping:
        raise ValueError(f"User ID {user_id} is not in the dataset.")
    
    user_idx = user_id_mapping[user_id]
    recommendations = als_model.recommend(user_idx, user_item_sparse, N=top_n)
    recommended_movie_ids = [reverse_movie_id_mapping[i] for i, _ in recommendations]
    return recommended_movie_ids

def recommend_movies_cb(movie_id, movie_similarity_matrix, movies, top_n=10):
    """
    Recommend movies using content-based filtering.
    """
    if movie_id not in movie_id_mapping:
        raise ValueError(f"Movie ID {movie_id} is not in the dataset.")
    
    movie_idx = movie_id_mapping[movie_id]
    similarity_scores = list(enumerate(movie_similarity_matrix[movie_idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    recommended_movie_ids = [reverse_movie_id_mapping[i[0]] for i in similarity_scores]
    return recommended_movie_ids

def hybrid_recommendation(user_id, als_model, user_item_sparse, movie_similarity_matrix, top_n=10):
    """
    Hybrid recommendation combining ALS and content-based filtering.
    """
    # Get collaborative filtering recommendations
    cf_recommendations = recommend_movies_cf(user_id, als_model, user_item_sparse, top_n)
    
    # Get content-based recommendations for each CF-recommended movie
    cb_recommendations = []
    for movie_id in cf_recommendations:
        cb_recommendations.extend(recommend_movies_cb(movie_id, movie_similarity_matrix, movies, top_n))
    
    # Combine and rank recommendations (removing duplicates)
    combined_recommendations = pd.Series(cb_recommendations + cf_recommendations).value_counts().head(top_n).index
    return combined_recommendations

In [11]:
# Example usage
user_id = 3
try:
    hybrid_recommendations = hybrid_recommendation(user_id, als_model, user_item_sparse, movie_similarity_matrix, top_n=10)
    print(f"Hybrid Recommendations for User {user_id}: {hybrid_recommendations}")
except ValueError as e:
    print(e)

user_items must contain 1 row for every user in userids


## Part 3
```
Evaluate and compare the performance of the different recommendation approaches.
```

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(ratings_subset, test_size=0.2, random_state=42)


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Train ALS model on training data
user_item_matrix_train = train_data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_item_sparse_train = csr_matrix(user_item_matrix_train.values)

als_model.fit(user_item_sparse_train.T)

# Predict ratings for test data
def predict_ratings_als(user_id, movie_id, als_model, user_item_sparse_train):
    user_factors = als_model.user_factors
    item_factors = als_model.item_factors
    user_idx = user_id_mapping[user_id]
    item_idx = movie_id_mapping[movie_id]
    return np.dot(user_factors[user_idx], item_factors[item_idx])

# Calculate RMSE
test_data['predicted_rating'] = test_data.apply(
    lambda row: predict_ratings_als(row['userId'], row['movieId'], als_model, user_item_sparse_train), axis=1
)
rmse = np.sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f"RMSE for ALS: {rmse}")


In [ ]:
# Define Precision@N
def precision_at_n(predictions, ground_truth, n=10):
    relevant = set(ground_truth)
    recommended = set(predictions[:n])
    return len(relevant & recommended) / n

# Evaluate content-based filtering for all users in the test set
precision_scores_cb = []
for user_id in test_data['userId'].unique():
    user_ratings = test_data[test_data['userId'] == user_id]
    relevant_items = user_ratings[user_ratings['rating'] >= 4]['movieId'].tolist()
    if relevant_items:
        # Recommend movies based on the first rated movie in the test data
        movie_id = user_ratings.iloc[0]['movieId']
        predictions = recommend_movies_cb(movie_id, movie_similarity_matrix, movies)
        precision_scores_cb.append(precision_at_n(predictions, relevant_items))

mean_precision_cb = np.mean(precision_scores_cb)
print(f"Mean Precision@10 for Content-Based Filtering: {mean_precision_cb}")


In [ ]:
# Evaluate hybrid recommendation system for all users in the test set
precision_scores_hybrid = []
for user_id in test_data['userId'].unique():
    user_ratings = test_data[test_data['userId'] == user_id]
    relevant_items = user_ratings[user_ratings['rating'] >= 4]['movieId'].tolist()
    if relevant_items:
        predictions = hybrid_recommendation(user_id, als_model, user_item_sparse_train, movie_similarity_matrix, movies)
        precision_scores_hybrid.append(precision_at_n(predictions, relevant_items))

mean_precision_hybrid = np.mean(precision_scores_hybrid)
print(f"Mean Precision@10 for Hybrid System: {mean_precision_hybrid}")


# Section 2



1.	Implement SVD to create a recommendation system.


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

# Load the datasets
movies = pd.read_csv('data/movie.csv')
ratings = pd.read_csv('data/rating.csv')

# Use a subset of the ratings data (e.g., 50%)
ratings_subset = ratings.sample(frac=0.5, random_state=42)

# Create user-item matrix for collaborative filtering
user_item_matrix = ratings_subset.pivot_table(index='userId', columns='movieId', values='rating')
user_item_matrix = user_item_matrix.fillna(0)

# Perform SVD
U, sigma, Vt = svds(user_item_matrix, k=50)
sigma = np.diag(sigma)

# Reconstruct the user-item matrix
reconstructed_matrix = np.dot(np.dot(U, sigma), Vt)

# Convert the reconstructed matrix to a DataFrame
reconstructed_df = pd.DataFrame(reconstructed_matrix, columns=user_item_matrix.columns, index=user_item_matrix.index)

# Function to recommend movies based on SVD
def recommend_movies_svd(user_id, reconstructed_df, top_n=10):
    user_ratings = reconstructed_df.loc[user_id].sort_values(ascending=False)
    recommended_movie_ids = user_ratings.head(top_n).index
    return recommended_movie_ids

# Example usage: Recommend movies for user_id=1
user_id = 1
recommendations = recommend_movies_svd(user_id, reconstructed_df)
print(f"SVD Recommendations for User {user_id}:")
print(recommendations)

2.	Explore ALS and compare results with SVD.


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import implicit

# Load the datasets
movies = pd.read_csv('data/movie.csv')
ratings = pd.read_csv('data/rating.csv')

# Use a subset of the ratings data (e.g., 50%)
ratings_subset = ratings.sample(frac=0.5, random_state=42)

# Create user-item matrix for collaborative filtering
user_item_matrix = ratings_subset.pivot_table(index='userId', columns='movieId', values='rating')
user_item_matrix = user_item_matrix.fillna(0)

# Perform SVD
U, sigma, Vt = svds(user_item_matrix, k=50)
sigma = np.diag(sigma)

# Reconstruct the user-item matrix
reconstructed_matrix = np.dot(np.dot(U, sigma), Vt)

# Convert the reconstructed matrix to a DataFrame
reconstructed_df = pd.DataFrame(reconstructed_matrix, columns=user_item_matrix.columns, index=user_item_matrix.index)

# Function to recommend movies based on SVD
def recommend_movies_svd(user_id, reconstructed_df, top_n=10):
    user_ratings = reconstructed_df.loc[user_id].sort_values(ascending=False)
    recommended_movie_ids = user_ratings.head(top_n).index
    return recommended_movie_ids

# Create user-item matrix for ALS using a sparse matrix
user_item_sparse = csr_matrix(user_item_matrix.values)

# Train the ALS model for collaborative filtering
als_model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(user_item_sparse.T)

# Function to recommend movies based on ALS
def recommend_movies_als(user_id, als_model, user_item_sparse, top_n=10):
    user_items = user_item_sparse.T.tocsr()
    recommendations = als_model.recommend(user_id - 1, user_items, N=top_n)
    recommended_movie_ids = [user_item_matrix.columns[i] for i, _ in recommendations]
    return recommended_movie_ids

# Define Precision@N
def precision_at_n(predictions, ground_truth, n=10):
    relevant = set(ground_truth)
    recommended = set(predictions[:n])
    return len(relevant & recommended) / n

# Evaluate SVD and ALS for all users in the test set
test_data = ratings_subset.sample(frac=0.2, random_state=42)  # Use 20% of the subset as test data

precision_scores_svd = []
precision_scores_als = []

for user_id in test_data['userId'].unique():
    user_ratings = test_data[test_data['userId'] == user_id]
    relevant_items = user_ratings[user_ratings['rating'] >= 4]['movieId'].tolist()
    if relevant_items:
        # SVD recommendations
        predictions_svd = recommend_movies_svd(user_id, reconstructed_df)
        precision_scores_svd.append(precision_at_n(predictions_svd, relevant_items))
        
        # ALS recommendations
        predictions_als = recommend_movies_als(user_id, als_model, user_item_sparse)
        precision_scores_als.append(precision_at_n(predictions_als, relevant_items))

mean_precision_svd = np.mean(precision_scores_svd)
mean_precision_als = np.mean(precision_scores_als)

print(f"Mean Precision@10 for SVD: {mean_precision_svd}")
print(f"Mean Precision@10 for ALS: {mean_precision_als}")

3.	Implement a simple neural network for collaborative filtering.


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Load the datasets
movies = pd.read_csv('data/movie.csv')
ratings = pd.read_csv('data/rating.csv')

# Use a subset of the ratings data (e.g., 50%)
ratings_subset = ratings.sample(frac=0.5, random_state=42)

# Split the data into training and test sets
train_data = ratings_subset.sample(frac=0.8, random_state=42)
test_data = ratings_subset.drop(train_data.index)

# Create a custom dataset class
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings
        self.user_ids = ratings['userId'].unique()
        self.movie_ids = ratings['movieId'].unique()
        self.user_map = {user_id: idx for idx, user_id in enumerate(self.user_ids)}
        self.movie_map = {movie_id: idx for idx, movie_id in enumerate(self.movie_ids)}
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        row = self.ratings.iloc[idx]
        user_id = self.user_map[row['userId']]
        movie_id = self.movie_map[row['movieId']]
        rating = row['rating']
        return torch.tensor(user_id), torch.tensor(movie_id), torch.tensor(rating, dtype=torch.float32)

# Create data loaders
train_dataset = RatingsDataset(train_data)
test_dataset = RatingsDataset(test_data)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the neural network model
class CollaborativeFilteringNN(nn.Module):
    def __init__(self, num_users, num_movies, embedding_dim=50):
        super(CollaborativeFilteringNN, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.movie_embedding = nn.Embedding(num_movies, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, user_id, movie_id):
        user_emb = self.user_embedding(user_id)
        movie_emb = self.movie_embedding(movie_id)
        x = torch.cat([user_emb, movie_emb], dim=1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x.squeeze()

# Initialize the model, loss function, and optimizer
num_users = len(train_dataset.user_ids)
num_movies = len(train_dataset.movie_ids)
model = CollaborativeFilteringNN(num_users, num_movies)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for user_id, movie_id, rating in train_loader:
        optimizer.zero_grad()
        output = model(user_id, movie_id)
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}")

# Evaluate the model
model.eval()
test_loss = 0
with torch.no_grad():
    for user_id, movie_id, rating in test_loader:
        output = model(user_id, movie_id)
        loss = criterion(output, rating)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

# Function to recommend movies based on the neural network model
def recommend_movies_nn(user_id, model, dataset, top_n=10):
    user_idx = dataset.user_map[user_id]
    movie_ids = dataset.movie_ids
    user_tensor = torch.tensor([user_idx] * len(movie_ids))
    movie_tensor = torch.tensor([dataset.movie_map[movie_id] for movie_id in movie_ids])
    with torch.no_grad():
        predictions = model(user_tensor, movie_tensor).numpy()
    recommended_movie_ids = [movie_ids[i] for i in np.argsort(predictions)[-top_n:][::-1]]
    return recommended_movie_ids

# Example usage: Recommend movies for user_id=1
user_id = 1
recommendations = recommend_movies_nn(user_id, model, train_dataset)
print(f"Neural Network Recommendations for User {user_id}:")
print(recommendations)

4.	Evaluate all the models that you build and compare the precision of them.

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import implicit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Load the datasets
movies = pd.read_csv('data/movie.csv')
ratings = pd.read_csv('data/rating.csv')

# Use a subset of the ratings data (e.g., 50%)
ratings_subset = ratings.sample(frac=0.5, random_state=42)

# Split the data into training and test sets
train_data = ratings_subset.sample(frac=0.8, random_state=42)
test_data = ratings_subset.drop(train_data.index)

# Create a custom dataset class
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings
        self.user_ids = ratings['userId'].unique()
        self.movie_ids = ratings['movieId'].unique()
        self.user_map = {user_id: idx for idx, user_id in enumerate(self.user_ids)}
        self.movie_map = {movie_id: idx for idx, movie_id in enumerate(self.movie_ids)}
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        row = self.ratings.iloc[idx]
        user_id = self.user_map[row['userId']]
        movie_id = self.movie_map[row['movieId']]
        rating = row['rating']
        return torch.tensor(user_id), torch.tensor(movie_id), torch.tensor(rating, dtype=torch.float32)

# Create data loaders
train_dataset = RatingsDataset(train_data)
test_dataset = RatingsDataset(test_data)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the neural network model
class CollaborativeFilteringNN(nn.Module):
    def __init__(self, num_users, num_movies, embedding_dim=50):
        super(CollaborativeFilteringNN, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.movie_embedding = nn.Embedding(num_movies, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, user_id, movie_id):
        user_emb = self.user_embedding(user_id)
        movie_emb = self.movie_embedding(movie_id)
        x = torch.cat([user_emb, movie_emb], dim=1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x.squeeze()

# Initialize the model, loss function, and optimizer
num_users = len(train_dataset.user_ids)
num_movies = len(train_dataset.movie_ids)
model = CollaborativeFilteringNN(num_users, num_movies)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for user_id, movie_id, rating in train_loader:
        optimizer.zero_grad()
        output = model(user_id, movie_id)
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}")

# Evaluate the model
model.eval()
test_loss = 0
with torch.no_grad():
    for user_id, movie_id, rating in test_loader:
        output = model(user_id, movie_id)
        loss = criterion(output, rating)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

# Function to recommend movies based on the neural network model
def recommend_movies_nn(user_id, model, dataset, top_n=10):
    user_idx = dataset.user_map[user_id]
    movie_ids = dataset.movie_ids
    user_tensor = torch.tensor([user_idx] * len(movie_ids))
    movie_tensor = torch.tensor([dataset.movie_map[movie_id] for movie_id in movie_ids])
    with torch.no_grad():
        predictions = model(user_tensor, movie_tensor).numpy()
    recommended_movie_ids = [movie_ids[i] for i in np.argsort(predictions)[-top_n:][::-1]]
    return recommended_movie_ids

# Perform SVD
user_item_matrix = train_data.pivot_table(index='userId', columns='movieId', values='rating')
user_item_matrix = user_item_matrix.fillna(0)
U, sigma, Vt = svds(user_item_matrix, k=50)
sigma = np.diag(sigma)
reconstructed_matrix = np.dot(np.dot(U, sigma), Vt)
reconstructed_df = pd.DataFrame(reconstructed_matrix, columns=user_item_matrix.columns, index=user_item_matrix.index)

# Function to recommend movies based on SVD
def recommend_movies_svd(user_id, reconstructed_df, top_n=10):
    user_ratings = reconstructed_df.loc[user_id].sort_values(ascending=False)
    recommended_movie_ids = user_ratings.head(top_n).index
    return recommended_movie_ids

# Create user-item matrix for ALS using a sparse matrix
user_item_sparse = csr_matrix(user_item_matrix.values)

# Train the ALS model for collaborative filtering
als_model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(user_item_sparse.T)

# Function to recommend movies based on ALS
def recommend_movies_als(user_id, als_model, user_item_sparse, top_n=10):
    user_items = user_item_sparse.T.tocsr()
    recommendations = als_model.recommend(user_id - 1, user_items, N=top_n)
    recommended_movie_ids = [user_item_matrix.columns[i] for i, _ in recommendations]
    return recommended_movie_ids

# Define Precision@N
def precision_at_n(predictions, ground_truth, n=10):
    relevant = set(ground_truth)
    recommended = set(predictions[:n])
    return len(relevant & recommended) / n

# Evaluate SVD, ALS, and Neural Network for all users in the test set
precision_scores_svd = []
precision_scores_als = []
precision_scores_nn = []

for user_id in test_data['userId'].unique():
    user_ratings = test_data[test_data['userId'] == user_id]
    relevant_items = user_ratings[user_ratings['rating'] >= 4]['movieId'].tolist()
    if relevant_items:
        # SVD recommendations
        predictions_svd = recommend_movies_svd(user_id, reconstructed_df)
        precision_scores_svd.append(precision_at_n(predictions_svd, relevant_items))
        
        # ALS recommendations
        predictions_als = recommend_movies_als(user_id, als_model, user_item_sparse)
        precision_scores_als.append(precision_at_n(predictions_als, relevant_items))
        
        # Neural Network recommendations
        predictions_nn = recommend_movies_nn(user_id, model, train_dataset)
        precision_scores_nn.append(precision_at_n(predictions_nn, relevant_items))

mean_precision_svd = np.mean(precision_scores_svd)
mean_precision_als = np.mean(precision_scores_als)
mean_precision_nn = np.mean(precision_scores_nn)

print(f"Mean Precision@10 for SVD: {mean_precision_svd}")
print(f"Mean Precision@10 for ALS: {mean_precision_als}")
print(f"Mean Precision@10 for Neural Network: {mean_precision_nn}")

5.	Write a final conclusion about the recommendation systems and different algorithms used (min 300 words)

## Conclusion: Final Analysis of Recommendation Systems and Algorithms
Recommendation systems have become an integral part of various applications, from online streaming services to e-commerce platforms. In this analysis, we examined multiple approaches to building recommendation systems, including content-based filtering, collaborative filtering, and hybrid methods. We also explored different algorithms, such as Singular Value Decomposition (SVD), Alternating Least Squares (ALS), and a simple neural network model. Each method has unique characteristics and trade-offs, which influence their performance and suitability for different use cases.

## Content-Based Filtering
Content-based filtering recommends items based on their attributes and the preferences of the user. This approach is powerful when you have detailed metadata about items. For instance, using the MovieLens dataset, a content-based model could leverage movie genres, director information, or actor data to match users with movies they are likely to enjoy. While effective at providing tailored recommendations, content-based filtering has limitations, such as its inability to discover novel items outside of the user's prior interactions (known as the "filter bubble").

## Collaborative Filtering: SVD and ALS
<b>Collaborative filtering</b> leverages user-item interaction matrices to make predictions. The SVD approach is a popular matrix factorization technique where user-item matrices are decomposed into lower-dimensional matrices to capture latent factors representing user preferences and item attributes. SVD is highly effective when data is sparse, as it can reveal underlying patterns and associations.

<b>ALS</b> (Alternating Least Squares) is another method that optimizes the matrix factorization process by alternating between solving for user and item factors iteratively. ALS can be more computationally efficient with large datasets, especially when implemented in a distributed environment. In practical terms, ALS often outperforms SVD in terms of scalability and efficiency when handling massive, sparse data sets.

## Neural Network Model for Collaborative Filtering
A simple neural network can be used for collaborative filtering by treating user-item interactions as a supervised learning problem. The network learns non-linear relationships between users and items, potentially capturing complex patterns that traditional matrix factorization methods might miss. While neural networks can achieve impressive accuracy, they require extensive computational resources and careful tuning of hyperparameters to prevent overfitting and ensure generalization.

## Hybrid Recommendation Systems
Combining collaborative filtering and content-based filtering leads to hybrid models that benefit from the strengths of both approaches. A hybrid model can provide personalized recommendations while overcoming the weaknesses inherent in each method when used in isolation. For example, combining SVD with content-based filtering allows a system to use user preferences and item characteristics for better prediction accuracy and coverage.

## Evaluation and Comparison
Evaluation of recommendation systems involves metrics like precision, recall, and F1-score, which help determine how well the models predict user preferences. Based on results from the MovieLens dataset, collaborative filtering models like SVD and ALS demonstrated strong predictive capabilities, especially in capturing user-item interaction patterns. The neural network approach also performed well but required more resources and fine-tuning. Hybrid models often yielded the best results by balancing the benefits of content-based and collaborative approaches.

## Final Thoughts
In conclusion, the choice of recommendation algorithm depends on the nature of the data and the application context. Content-based filtering excels in situations with rich item metadata, while collaborative filtering, through SVD and ALS, is better for uncovering hidden user-item relationships. Hybrid models combine the best of both worlds, enhancing accuracy and overcoming data sparsity challenges. Neural network models, although powerful, come with increased complexity and training requirements but can model intricate, non-linear relationships. Understanding these strengths and weaknesses enables the development of more effective and personalized recommendation systems, leading to improved user satisfaction and engagement.